## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/weatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sisimiut,GL,66.9395,-53.6735,26.98,56,100,11.07,overcast clouds
1,1,Barrow,US,71.2906,-156.7887,4.48,78,100,14.14,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,-19.71,84,90,11.50,light snow
3,3,Kapaa,US,22.0752,-159.3190,74.19,99,0,7.43,clear sky
4,4,Faya,SA,18.3851,42.4509,53.71,81,37,4.21,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
12,12,Vaini,TO,-21.2000,-175.2000,80.76,89,75,11.50,shower rain
31,31,Butaritari,KI,3.0707,172.7902,82.36,62,100,8.99,overcast clouds
37,37,Georgetown,MY,5.4112,100.3354,80.89,80,20,1.99,few clouds
43,43,Lorengau,PG,-2.0226,147.2712,80.78,77,18,7.18,few clouds
63,63,Arraial Do Cabo,BR,-22.9661,-42.0278,82.36,65,75,11.50,broken clouds
69,69,Yulara,AU,-25.2406,130.9889,82.24,17,5,8.05,clear sky
76,76,Karratha,AU,-20.7377,116.8463,84.76,65,55,11.92,light rain
89,89,San Vicente,AR,-26.6167,-54.1333,85.50,46,27,1.77,light rain
99,99,Chuy,UY,-33.6971,-53.4616,86.83,50,100,3.31,light rain
105,105,Samarai,PG,-10.6167,150.6667,80.87,80,100,7.31,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any')
preferred_cities_df.count()

City_ID                86
City                   86
Country                86
Lat                    86
Lng                    86
Max Temp               86
Humidity               86
Cloudiness             86
Wind Speed             86
Current Description    86
dtype: int64

In [7]:
# 4b. Create a new DataFrame that doesn’t have empty rows.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Vaini,TO,80.76,-21.2000,-175.2000,
31,Butaritari,KI,82.36,3.0707,172.7902,
37,Georgetown,MY,80.89,5.4112,100.3354,
43,Lorengau,PG,80.78,-2.0226,147.2712,
63,Arraial Do Cabo,BR,82.36,-22.9661,-42.0278,
69,Yulara,AU,82.24,-25.2406,130.9889,
76,Karratha,AU,84.76,-20.7377,116.8463,
89,San Vicente,AR,85.50,-26.6167,-54.1333,
99,Chuy,UY,86.83,-33.6971,-53.4616,
105,Samarai,PG,80.87,-10.6167,150.6667,


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Vaini,TO,80.76,shower rain,-21.2000,-175.2000,
31,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,
37,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
43,Lorengau,PG,80.78,few clouds,-2.0226,147.2712,
63,Arraial Do Cabo,BR,82.36,broken clouds,-22.9661,-42.0278,
69,Yulara,AU,82.24,clear sky,-25.2406,130.9889,
76,Karratha,AU,84.76,light rain,-20.7377,116.8463,
89,San Vicente,AR,85.50,light rain,-26.6167,-54.1333,
99,Chuy,UY,86.83,light rain,-33.6971,-53.4616,
105,Samarai,PG,80.87,overcast clouds,-10.6167,150.6667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.count()

City                   86
Country                86
Max Temp               86
Current Description    86
Lat                    86
Lng                    86
Hotel Name             86
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(how='any')
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Vaini,TO,80.76,shower rain,-21.2000,-175.2000,Keleti Beach Resort
31,Butaritari,KI,82.36,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
37,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
43,Lorengau,PG,80.78,few clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
63,Arraial Do Cabo,BR,82.36,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
69,Yulara,AU,82.24,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
76,Karratha,AU,84.76,light rain,-20.7377,116.8463,ibis Styles Karratha
89,San Vicente,AR,85.50,light rain,-26.6167,-54.1333,Complejo Turístico Farelluk
99,Chuy,UY,86.83,light rain,-33.6971,-53.4616,Nuevo Hotel Plaza
105,Samarai,PG,80.87,overcast clouds,-10.6167,150.6667,Nuli Sapi


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptio</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))